In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import numpy as np
import jax
!pip install mycolorpy
from scipy.stats import multivariate_normal

  Preparing metadata (setup.py) ... done
  Created wheel for mycolorpy: filename=mycolorpy-1.5.1-py3-none-any.whl size=3853 sha256=b152fd1deb7bf52071f1049683c6949531e604d443d9170820c8119e530fbc16
  Stored in directory: /root/.cache/pip/wheels/d6/1d/04/f8bfb8b04e3daba330b3f377c041b89304271ff98fb6704946
Successfully built mycolorpy


In [2]:
from IPython.utils.sysinfo import num_cpus

# numero di cluster
K=10

# dimensione del campione
d=2

# numero di sample
N=2000

seed=2021
key = random.PRNGKey(seed)
# vectors of mean of clusters
sigma=2
mu=random.normal(key,(K,d))*sigma

seed=2023
# cluster assignment
key = random.PRNGKey(seed)
c=random.categorical(key,(1/K)*jnp.ones(shape=(K,)),axis=0,shape=(N,))
C=np.zeros(shape=(N,K))
for i in range(N):
  C[i,c[i]]=1
C=jnp.array(C)

# Data
X=jnp.matmul(C,mu)+random.normal(key,(N,d))

In [ ]:
'''
@jit
def sample_phi(data,mu):
  N=data.shape[0]
  K=mu.shape[0]
  for i in jnp.arange(N):
    for k in jnp.arange(K):
      post_c[i,k]=jnp.exp(-0.5*jnp.matmul(data[i,]-mu[k,],(data[i,]-mu[k,]).T))
  post_c/=jnp.sum(post_c)
  phi_value=random.categorical(prob=post_c,(N,))
  phi=jnp.zeros((N,K))
  for i in jnp.arange(N):
    phi[i,phi_value[i]]=1
  return phi

@jit
def sample_mu(data,phi,sigma):
  K=phi.shape[1]
  mu=jnp.zeros((K,d))
  n_cluster=jnp.sum(phi,axis=0,keepdims=True)
  for i in jnp.arange(K):
    sample_size=n_cluster[i]
    sample_mean=jnp.matmul(phi[,i].T,data)/sample_size
    post_prec=sample_size+1/sigma**2
    post_mean=sample_mean*sample_size/post_prec
    mu[i,]=random.normal(post_mean,jnp.sqrt(1/post_prec),(1,d))
  return mu

@jit
def gibbs(data,K,sigma,niter):
  key = random.PRNGKey(seed)
  mu=random.normal(key,(K,d))
  phi=sample_phi(data,mu)
  stored_mu=jnp.zeros((niter,K,d))
  stores_phi=jnp.zeros((niter,N,K))
  stored_mu[1,]=mu
  stored_phi[1,]=phi
  for i in jnp.arange(niter):
    mu=sample_mu(data,phi,sigma)
    phi=sample_phi(data,mu)
    stored_mu[i,]=mu
    stored_phi[i,]=phi

  m=jnp.mean(stored_mu,axis=0)
  s2=jnp.mean(jnp.var(stored_mu,axis=0),axis=1)
  phi=
  return m,s2

'''

In [3]:
n_cluster = np.sum(C, axis = 0)
m_post=1/(1+1/sigma**2)/np.matmul(n_cluster.reshape((K,1)),np.ones(shape=(1,d)))*np.matmul(C.T,X)
s2_post=1/(1+1/sigma**2)/n_cluster

In [14]:

def gibbs_gaussian_mixture(data, num_clusters, sigma, num_iterations):
    # Data dimensions
    N, D = data.shape

    # Initialization
    cluster_assignments = np.random.choice(num_clusters, N)
    cluster_means_samples = np.zeros((num_iterations, num_clusters, D))
    cluster_variances_samples = np.zeros((num_iterations, num_clusters))
    cluster_probabilities_samples = np.zeros((num_iterations, N, num_clusters))

    # Initialize cluster means and variances
    cluster_means = np.random.multivariate_normal(mean=np.mean(data, axis=0), cov=np.eye(D), size=num_clusters)
    cluster_variances = np.ones((num_clusters, D))

    # Gibbs Sampler
    for iteration in range(num_iterations):
        # Update cluster assignments
        for i in range(N):
            # Compute probabilities for each cluster
            likelihoods = [multivariate_normal.pdf(data[i,], mean=cluster_means[k,], cov=np.diag(cluster_variances[k,])) for k in range(num_clusters)]

            # Update cluster assignment using categorical prior
            probabilities = likelihoods / np.sum(likelihoods)
            cluster_assignments[i] = np.random.choice(num_clusters, p=probabilities)

        # Update cluster means and variances
        for k in range(num_clusters):
            cluster_size = np.sum(cluster_assignments == k)
            prior_mean = np.zeros(D)
            prior_covariance = sigma**2 * np.eye(D)

            # Compute posterior parameters
            posterior_covariance = np.linalg.inv(prior_covariance + (cluster_size / np.var(data[cluster_assignments == k,], axis=0)) * np.eye(D))
            posterior_mean = posterior_covariance @ (1/sigma**2 * prior_mean + (cluster_size / np.var(data[cluster_assignments == k,], axis=0)) * np.sum(data[cluster_assignments == k,], axis=0))

            # Update cluster means
            cluster_means[k, :] = np.random.multivariate_normal(mean=posterior_mean, cov=posterior_covariance)

            # Store samples
            cluster_means_samples[iteration, k, :] = cluster_means[k, :]
            cluster_variances_samples[iteration, k] = np.mean(np.var(data[cluster_assignments == k,], axis=0)) / cluster_size

        # Compute cluster probabilities for each data point
        for i in range(N):
            for k in range(num_clusters):
                likelihood = multivariate_normal.pdf(data[i,], mean=cluster_means[k,], cov=np.diag(cluster_variances[k]))
                cluster_probabilities_samples[iteration, i, k] = likelihood / np.sum([multivariate_normal.pdf(data[i,], mean=cluster_means[j,], cov=np.diag(cluster_variances[j])) for j in range(num_clusters)])

    # Compute final means and variances based on collected samples
    final_cluster_means = np.mean(cluster_means_samples, axis=0)
    final_cluster_variances = np.mean(cluster_variances_samples, axis=0)

    # Compute final cluster probabilities for each data point
    final_cluster_probabilities = np.mean(cluster_probabilities_samples, axis=0)

    return final_cluster_means, final_cluster_variances, final_cluster_probabilities


In [15]:
n_iterations=100
result_means, result_variances, result_probs = gibbs_gaussian_mixture(X, K, sigma, n_iterations)

print("Final Cluster Means:")
print(result_means)

print("\nFinal Cluster Variances:")
print(result_variances)

print("\nCluster Probabilities:")
print(result_probs)

LinAlgError: ignored

In [ ]:
m_post,s2_post